# 論文読解

以下の論文を読み問題に答えてください。CNNを使った物体検出（Object Detection）の代表的な研究です。


[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99


https://arxiv.org/pdf/1506.01497.pdf

# 問題
それぞれについてJupyter Notebookにマークダウン形式で記述してください。


(1) 物体検出の分野にはどういった手法が存在したか。


(2) Fasterとあるが、どういった仕組みで高速化したのか。


(3) One-Stageの手法とTwo-Stageの手法はどう違うのか。


(4) RPNとは何か。


(5) RoIプーリングとは何か。


(6) Anchorのサイズはどうするのが適切か。


(7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。


(8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

### 条件
- 答える際は論文のどの部分からそれが分かるかを書く。
- 必要に応じて先行研究（引用されている論文）も探しにいく。最低2つは他の論文を利用して回答すること。
- 論文の紹介記事を見ても良い。ただし、答えは論文内に根拠を探すこと。

## (1) 物体検出の分野にはどういった手法が存在したか。

R-CNN(Regional CNN)やSPPnet(Spatial Pyramid Pooling Networks)など領域候補を摘出する手法が存在している。<br>
Recent advances in object detection are driven by the success of region proposal methods (e.g., [4]) and region-based convolutional neural networks (RCNNs) [5]. 引用元 --R. Girshick, “Fast R-CNN,” in IEEE International Conference on Computer Vision (ICCV), 2015

## (2) Fasterとあるが、どういった仕組みで高速化したのか。

RPNsは最新の物体検出ネットワークと畳み込み層を共有する。これにより、test-time時における、領域候補を計算するための限界コストを小さくすることが可能である。（画像あたり１０ミリ秒）<br>
*引用元: p.1 (...we introduce novel Region Proposal Networks (RPNs) that share convolutional layers with state-of-the-art object detection networks [1], [2]. By sharing convolutions at test-time, the marginal cost for computing proposals is small (e.g., 10ms per image).)

マルチスケーリングや多数のフィルタサイズを使用するのではなく、anchor boxesを使用することで、多くのサンプルやフィルタの繰り返し処理が不要な仕組みで高速化。<br>
*引用元：p1-p2 (In contrast to prevalent methods [8], [9], [1], [2] that use pyramids of images (Figure 1, a) or pyramids of filters (Figure 1, b), we introduce novel “anchor” boxes that serve as references at multiple scales and aspect ratios.)

## (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

- One-Stage: 特徴マップ上をフィルタで探索し領域候補の摘出及び検出物体の分類を同時に行う
- Two-Stage: 領域候補と分類を直列に連結された個別のモデルが行う
<br>

*引用元：p10 (The OverFeat paper [9] proposes a detection method that uses regressors and classifiers on sliding windows over convolutional feature maps. OverFeat is a one-stage, class-specific detection pipeline, and ours is a two-stage cascade consisting of class-agnostic pro- posals and class-specific detections.)

## (4) RPNとは何か。

Region Proposal Networks. 画像上に作成された領域提案の中から、物体が含まれそうな領域を検出するモデル。推定された各領域にはobjectness score（物体が含まれる確率）によって評価される。<br>
*引用元：p3 (A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score.)

## (5) RoIプーリングとは何か。

RoI Poolingとは、Fast R-CNNで実装された機構であり、抽出されたn個のRoI (Region of Interest)を固定されたサイズHxWのfeature mapにMaxPoolingされる。<br>
*引用元：R. Girshick, Fast R-CNN p1より(The RoI pooling layer uses max pooling to convert the features inside any valid region of interest into a small feature map with a fixed spatial extent of H × W (e.g., 7 × 7), where H and W are layer hyper-parameters that are inde- pendent of any particular RoI)

一方Faster R-CNNでは、Fast版と違ってRoIが固定の大きさなため、プーリング後のfeature map上でも全RoIが同サイズである。異なるRoIサイズに対応するため、RoIサイズひとつ毎にfeature mapを生成し、ひとつのfeature mapにつき回帰モジュール (regressors)を別に用意する。この手法によって、異なるRoIサイズ間にて重みが共有されることを回避できる。<br>
*引用元：p5 (In our formula- tion, the features used for regression are of the same spatial size (3 × 3) on the feature maps. To account for varying sizes, a set of k bounding-box regressors are learned. Each regressor is responsible for one scale and one aspect ratio, and the k regressors do not share weights.)

参考：　[Qiita - 物体検出についての歴史まとめ(1)](https://qiita.com/mshinoda88/items/9770ee671ea27f2c81a9)

入力リストからすべての関心領域について、それに対応する入力特徴マップのセクションを取り、それを予め定義されたサイズ（例えば、7×7）にスケールします。スケーリングは次の方法で行います。

領域候補を同じサイズのセクションに分割します（その数は出力の次元と同じ）
各セクションで最大値を見つける
これらの最大値を出力バッファにコピーする
その結果、サイズの異なる長方形のリストから、固定サイズの対応する特徴マップのリストをすばやく取得できます。RoIプーリング出力の次元は、実際には入力特徴マップのサイズや領域提案のサイズに依存しないことに注意。領域候補を分割するセクションの数だけによって決定されます。

RoIプーリングのメリットの1つは処理速度です。フレームに複数の物体候補がある場合（通常はたくさんある）、それらのすべてに対して同じ入力特徴マップを使用できます。ネットワーク処理の初期段階での畳み込み計算は非常にコストがかかるので、このアプローチは時間を大幅に節約できます。

実際の動作を見てみましょう。今、8×8の単一の特徴マップの1つの関心領域に対して2×2の出力サイズでRoIプーリングを実行してみます。入力特徴マップは次のようになります。

![](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F199265%2F03b3cc1c-7716-62e1-c6f8-ede44cde8a9a.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=c33488cb420e71b6cde37d0e2cfd8972RoIプーリングは実際に何をするのかを見ていきます。
)

領域候補（左上、右下座標）の情報（0、3）、（7,8）があるとします。

![](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F199265%2F3da5ff35-d86e-da29-d1c8-4d04c2f43250.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=2d5ea62ea434e9b7bc6f7436e3e357a4)

通常は、それぞれに複数の特徴マップと複数の候補がありますが、この例では単純化しています。出力サイズが2×2であるため、（2×2）セクションに分割すると、次のようになります。

![](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F199265%2F7d5d15da-1a22-949f-4460-6ffdadfe280f.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=1ddd0bc5679a5c05a04f1b778f3abce3)

関心領域のサイズは、プールセクションの数によって完全に割り切れる必要はないことに注意。
この場合、RoIは7×5で、プールセクションは2×2です。各セクションの最大値は次のとおりです。

![](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F199265%2Fefa2b193-3d7c-ca0c-cb48-463540799462.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=671c65b72dc50b17637d8a4929c74813)

アニメーションの形で提示された例を示しておきます。

![](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F199265%2F6af0ded3-29b2-7035-d935-440a0232b6ae.gif?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=3e4399813d86f64fb03a18ecb11e4cc7)

## (6) Anchorのサイズはどうするのが適切か。

3つのanchor size（1282, 2562, 5122）と１つまたは３つのアスペクト比（2:1, 1:1, 1:2）が採用データセット（PASCAL VOC 2007）に対して、最も高いmAP値を示した。アスペクト比の使用数はmAPに影響がなく、anchor sizeとアスペクト比に相関があると考察されている。ただ、モデルの汎化性を保つため、両パラメータを採用されている。<br>
*引用元：p6 (For anchors, we use 3 scales with box areas of 1282, 2562, and 5122 pixels, and 3 aspect ratios of 1:1, 1:2, and 2:1. These hyper-parameters are not carefully cho- sen for a particular dataset, and we provide ablation experiments on their effects in the next section.); p9 (The mAP is higher if using 3 scales (with 1 aspect ratio) or 3 aspect ratios (with 1 scale), demonstrating that using anchors of multiple sizes as the regression references is an effective solution. Using just 3 scales with 1 aspect ratio (69.8%) is as good as using 3 scales with 3 aspect ratios on this dataset, suggesting that scales and aspect ratios are not disentangled dimensions for the detection accuracy. But we still adopt these two dimensions in our designs to keep our system flexible.)

## (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

本研究ではPASCAL VOC 2007＋2012、及びMicrosoft COCOのデータセットが使用されている。両データセットとも過去にコンペで提供されたデータセットであり、PASCAL VOCは20種のカテゴリーを含むtrainvalデータが5000点、testデータが5000点あり、Microsoft COCOは80種のカテゴリーを含むtrain80,000点、validation40,000点、test20,000点が使用されている。先行研究との比較にはCOCOデータセットにて検証しており、testデータに対してFast R-CNNの評価がmAP@0.5: 39.3とmAP@[.5, .95]: 19.3だったところ、Faster R-CNNだとmAP@0.5: 42.1とmAP@[.5, .95]: 21.5であった。<br>
*引用元：p2 (We comprehensively evaluate our method on the PASCAL VOC 2007 detection benchmark [11]. This dataset consists of about 5k trainval images and 5k test images over 20 object categories.); p10 (This dataset involves 80 object categories. We experiment with the 80k images on the training set, 40k images on the validation set, and 20k images on the test-dev set. We evaluate the mAP averaged for IoU ∈ [0.5 : 0.05 : 0.95] (COCO’s standard metric, simply denoted as mAP@[.5, .95]) and mAP@0.5 (PASCAL VOC’s metric))

## (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

YOLOについて

YOLO(You only Look Once)
YOLOは予め画像全体をグリッド分割しておき、各領域ごとに物体のクラスとbounding boxを求める、という方法を採用しています。
CNNのアーキテクチャがシンプルになったため、Faster R-CNNに識別精度は少し劣りますが45-155FPSの検出速度を達成しています。
またスライディングウィンドウや領域候補(Region Proposal)を使った手法と違い、1枚の画像の全ての範囲を学習時に利用するため、周辺のコンテクストも同時に学習することができます。

これにより、背景の誤検出を抑えることができるようになり、背景の誤検出はFast R-CNNの約半分の抑えることが出来ました。

YOLO 詳細
画像全体をS×Sのグリッドに分割し、各グリッドはN個の bounding box と confidence を予測します。confidence は、

bounding box の精度と
bounding box が実際にオブジェクトを含むかどうか（クラスに関係なく）
を反映します。YOLO はまた、トレーニング中の各クラスの各 bounding box の分類スコアを予測します。両方のクラスを組み合わせて、各クラスが予測される bounding box に存在する確率を計算することができます。

合計SxSxN 個の bounding box が予測されますが、多くの bounding box は confidence スコアが低く、しきい値を30％と設定すると、下の例に示すようにほとんどのものが削除されます。

![](https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F199265%2F5faea925-50aa-4991-2775-f9810f40393b.jpeg?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=1fae0b6f3d3ed11576c6cb5583dc4e91)

YOLOは超高速であり、リアルタイムで実行可能です。

従来手法では、生成された領域候補(region proposals)のみを見ていたのに対して、
YOLOは画像全体を一度のみ見ることです。
したがって、背景の誤検出回避に役立ちます。

YOLOの欠点
分割されたグリッドサイズは固定かつ、グリッド内で識別できるクラスは1つ
検出できる物体の数は2つ
という制約があるため、 グリッド内に大量の物体がある場合に弱くなります。

